In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import wave
import pylab
from pathlib import Path
from scipy import signal
from scipy.io import wavfile
from sklearn.metrics import confusion_matrix
import itertools
import os
import IPython.display as ipd

import scipy.io.wavfile as wav
import wave
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings("ignore")
import IPython.display as ipd
import scipy.io.wavfile as wav
import pylab
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from sklearn.metrics import f1_score
from tensorflow.keras.models import Sequential, Model, load_model

In [ ]:
import tensorflow as tf
tf.get_logger().setLevel('INFO')

In [ ]:
input_dir = '/kaggle/input/free-spoken-digits/free-spoken-digit-dataset-master/recordings/'
output_dir = '/kaggle/working/'

In [ ]:
parent_list = os.listdir(input_dir)
for i in range(10):
    print(parent_list[i])

### Signal visualization

In [ ]:
for i in range(5): 
    signal_wave = wave.open(os.path.join(input_dir, parent_list[i]), 'r')
    sample_rate = 16000
    sig = np.frombuffer(signal_wave.readframes(sample_rate), dtype=np.int16)

    plt.figure(figsize=(12,12))
    plot_a = plt.subplot(211)
    plot_a.set_title(parent_list[i])
    plot_a.plot(sig)
    plot_a.set_xlabel('sample rate * time')
    plot_a.set_ylabel('energy')

    plot_b = plt.subplot(212)
    plot_b.specgram(sig, NFFT=1024, Fs=sample_rate, noverlap=900)
    plot_b.set_xlabel('Time')
    plot_b.set_ylabel('Frequency')

plt.show()

In [ ]:
SIZE = 256
BATCH_SIZE = 32
channels = 3
kernel = 4
stride = 1
pool = 2
for filename in os.listdir(input_dir):
    if "wav" in filename:
        file_path = os.path.join(input_dir, filename)
        target_dir = f'class_{filename[0]}'             
        dist_dir = os.path.join(output_dir, target_dir)
        file_dist_path = os.path.join(dist_dir, filename)
        if not os.path.exists(file_dist_path + '.png'):
            if not os.path.exists(dist_dir):
                os.mkdir(dist_dir)                
            frame_rate, data = wav.read(file_path)
            signal_wave = wave.open(file_path)
            sig = np.frombuffer(signal_wave.readframes(frame_rate), dtype=np.int16)
            fig = plt.figure()
            plt.specgram(sig, NFFT=1024, Fs=frame_rate, noverlap=900)
            plt.axis('off')
            fig.savefig(f'{file_dist_path}.png', dpi=fig.dpi)
            plt.close()

In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                             batch_size=BATCH_SIZE,
                                             validation_split=0.2,
                                             directory=output_dir,
                                             shuffle=True,
                                             color_mode='rgb',
                                             image_size=(SIZE, SIZE),
                                             subset="training",
                                             seed=0);

valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                             batch_size=BATCH_SIZE,
                                             validation_split=0.2,
                                             directory=output_dir,
                                             shuffle=True,
                                             color_mode='rgb',
                                             image_size=(SIZE, SIZE),
                                             subset="validation",
                                             seed=0);

#### Found 3000 files belonging to 11 classes.
#### Using 2400 files for training.
#### Found 3000 files belonging to 11 classes.
#### Using 600 files for validation.

In [ ]:
plt.figure(figsize=(12, 12));
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1);
        plt.imshow(images[i].numpy().astype("uint8"));
        plt.title(int(labels[i]));
        plt.axis("off");
plt.show();

### CNN Model

In [ ]:
class_names = train_dataset.class_names
num_classes = len(class_names)
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = valid_dataset.cache().prefetch(buffer_size=AUTOTUNE)

model = Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=(SIZE, SIZE, channels)),
    layers.Conv2D(16, kernel, stride, activation='relu'),
    layers.MaxPool2D(pool),
    layers.Conv2D(32, kernel, stride, activation='relu'),
    layers.MaxPool2D(pool),
    layers.Conv2D(64, kernel, stride, activation='relu'),
    layers.MaxPool2D(pool),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])
model.summary()

In [ ]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='checkpoint.h5',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
model.compile(optimizer='adam', 
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                  metrics=['accuracy'])

In [ ]:
epochs = 15
history = model.fit(
    train_ds, epochs=epochs, callbacks=model_checkpoint_callback, validation_data=val_ds,
);

In [ ]:
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values)+1)

plt.figure(figsize=(8,6))
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(acc_values)+1)

plt.figure(figsize=(8,6))
plt.plot(epochs, acc_values, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc_values, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

### LSTM Model


In [ ]:
import librosa,librosa.display
from tqdm.notebook import tqdm

In [ ]:
path='../input/free-spoken-digits/free-spoken-digit-dataset-master/recordings/0_george_0.wav'
dir_path='../input/free-spoken-digits/free-spoken-digit-dataset-master/recordings/'

In [ ]:
raw_data,framerate=librosa.load(path)
raw_data,framerate

In [ ]:
data=pd.DataFrame(columns=['raw_data','duration','digit'])
dir_path='../input/free-spoken-digits/free-spoken-digit-dataset-master/recordings/'
for i in tqdm(os.listdir(dir_path)):
        raw_data,frame_rate=librosa.load(dir_path+i)
        duration=librosa.get_duration(raw_data,frame_rate)
        data.loc[len(data.index)]=[raw_data,duration,i.split('_')[0]]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data[['raw_data','duration']],data['digit'], test_size=0.3, random_state=45)

In [ ]:
max_length=20366
X_train=tf.keras.preprocessing.sequence.pad_sequences(X_train['raw_data'],maxlen=max_length, dtype='float32')
X_test=tf.keras.preprocessing.sequence.pad_sequences(X_test['raw_data'],maxlen=max_length, dtype='float32')

In [ ]:
X_train.shape

### convert the signals to spectogram

In [ ]:
def convert_to_spectrogram(raw_data):
    spect = librosa.feature.melspectrogram(y=raw_data, n_mels=64) # n_mels as output shape
    mel_spect = librosa.power_to_db(S=spect, ref=np.max)
    return mel_spect

In [ ]:
X_train_spectrogram=np.array([convert_to_spectrogram(np.array([float(i) for i in X_train[k] ])) for k in range(len(X_train)) ])
X_test_spectrogram=np.array([convert_to_spectrogram(np.array([float(i) for i in X_test[k] ])) for k in range(len(X_test)) ])
X_train_spectrogram.shape

In [ ]:
input_layer=Input(shape=(64,40), dtype=np.float32,name='input_layer')
lstm=LSTM(500,name='lstm_layer',return_sequences=True)(input_layer)
d1=Dense(120,activation='relu',name='dense1')(tf.math.reduce_mean(lstm, 2))
d2=Dense(60,activation='relu',name='dense2')(d1)
d3=Dense(60,activation='softmax',name='dense3')(d2)
model = Model(inputs=input_layer, outputs=d3)
model.summary()

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=path, monitor='val_micro_f1',  verbose=1, save_best_only=True, mode='max')
opt= tf.keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy' ,metrics=['accuracy'])

In [ ]:
lstm_h=model.fit(X_train_spectrogram,y_train.astype('int')\
           ,validation_data=(X_test_spectrogram,y_test.astype('int'))\
           ,batch_size=32,epochs=15\
           ,callbacks=[checkpoint])

In [ ]:
acc = lstm_h.history['accuracy']
val_acc = lstm_h.history['val_accuracy']

loss = lstm_h.history['loss']
val_loss = lstm_h.history['val_loss']


plt.figure(figsize=(12, 12))

plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy', color='r')
plt.plot(val_acc, label='Validation Accuracy', color='b')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='lower right', fontsize=13)
plt.ylabel('Accuracy', fontsize=16, weight='bold')
plt.title('LSTM - Training & Validation Acc.', fontsize=16, weight='bold')


plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss', color='r')
plt.plot(val_loss, label='Validation Loss', color='b')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='upper right', fontsize=13)
plt.ylabel('Cross Entropy', fontsize=16, weight='bold')
plt.title('LSTM - Training & Validation Loss', fontsize=15, weight='bold')
plt.xlabel('Epoch', fontsize=15, weight='bold')

plt.show()